# Energy Consumption Predictions with Bayesian LSTMs in PyTorch

Author: Pawarit Laosunthara



# **Important Note for GitHub Readers:**
Please click the **Open in Colab** button above in order to view all **interactive visualizations**.

This notebook demonstrates an implementation of an (Approximate) Bayesian Recurrent Neural Network in PyTorch, originally inspired by the *Deep and Confident Prediction for Time Series at Uber* (https://arxiv.org/pdf/1709.01907.pdf)

<br>

In this approach, Monte Carlo dropout is used to **approximate** Bayesian inference, allowing our predictions to have explicit uncertainties and confidence intervals. This property makes Bayesian Neural Networks highly appealing to critical applications requiring uncertainty quantification.
The *Appliances energy prediction* dataset used in this example is from the UCI Machine Learning Repository (https://archive.ics.uci.edu/ml/datasets/Appliances+energy+prediction)


**Note:** this notebook purely serves to demonstrate the implementation of Bayesian LSTMs (Long Short-Term Memory) networks in PyTorch. Therefore, extensive data exploration and feature engineering is not part of the scope of this investigation.

# Preliminary Data Wrangling

**Selected Columns:**

For simplicity and speed when running this notebook, only temporal and autoregressive features are used.

- date time year-month-day hour:minute:second, sampled every 10 minutes \
- Appliances, energy use in Wh for the corresponding 10-minute timestamp \
- day_of_week, where Monday corresponds to 0 \
- hour_of_day


In [1]:
import pandas as pd

In [6]:
energy_df = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/00374/energydata_complete.csv')

energy_df['date'] = pd.to_datetime(energy_df['date'])

energy_df['month'] = energy_df['date'].dt.month.astype(int)
energy_df['day_of_month'] = energy_df['date'].dt.day.astype(int)

# day_of_week=0 corresponds to Monday
energy_df['day_of_week'] = energy_df['date'].dt.dayofweek.astype(int)
energy_df['hour_of_day'] = energy_df['date'].dt.hour.astype(int)

selected_columns = ['date', 'day_of_week', 'hour_of_day', 'Appliances']
energy_df = energy_df[selected_columns]

In [3]:
energy_df.head()

,date,day_of_week,hour_of_day,Appliances
0,2016-01-11 17:00:00,0,17,60
1,2016-01-11 17:10:00,0,17,60
2,2016-01-11 17:20:00,0,17,50
3,2016-01-11 17:30:00,0,17,50
4,2016-01-11 17:40:00,0,17,60


## Time Series Transformations

1. The dataset is to be re-sampled at an hourly rate for more meaningful analytics.

2. To alleviate exponential effects, the target variable is log-transformed as per the Uber paper.

3. For simplicity and speed when running this notebook, only temporal and autoregressive features, namely `day_of_week`, `hour_of_day`, \
and previous values of `Appliances` are used as features

In [7]:
import numpy as np

resample_df = energy_df.set_index('date').resample('1H').mean()


In [9]:
resample_df.head(7)

,day_of_week,hour_of_day,Appliances
date,,,
2016-01-11 17:00:00,0.0,17.0,55.000000
2016-01-11 18:00:00,0.0,18.0,176.666667
2016-01-11 19:00:00,0.0,19.0,173.333333
2016-01-11 20:00:00,0.0,20.0,125.000000
2016-01-11 21:00:00,0.0,21.0,103.333333
2016-01-11 22:00:00,0.0,22.0,266.666667
2016-01-11 23:00:00,0.0,23.0,56.666667


In [11]:
resample_df['date'] = resample_df.index
resample_df['log_energy_consumption'] = np.log(resample_df['Appliances'])

datetime_columns = ['date', 'day_of_week', 'hour_of_day']
target_column = 'log_energy_consumption'

feature_columns = datetime_columns + ['log_energy_consumption']

# For clarity in visualization and presentation, 
# only consider the first 150 hours of data.
resample_df = resample_df[feature_columns]

In [ ]:
import plotly.express as px

plot_length = 150
plot_df = resample_df.copy(deep=True).iloc[:plot_length]
plot_df['weekday'] = plot_df['date'].dt.day_name()

fig = px.line(plot_df,
              x="date",
              y="log_energy_consumption", 
              color="weekday", 
              title="Log of Appliance Energy Consumption vs Time")
fig.show()

# Prepare Training Data

For this example, we will use sliding windows of 10 points per each window (equivalent to 10 hours) to predict each next point. The window size can be altered via the `sequence_length` variable.

Min-Max scaling has also been fitted to the training data to aid the convergence of the neural network. 

In [12]:
from sklearn.preprocessing import MinMaxScaler

sequence_length = 10
output_length = 1


def create_sliding_window(data, sequence_length, stride=1):
    X_list, y_list = [], []
    for i in range(len(data)):
      if (i + sequence_length) < len(data):
        X_list.append(data.iloc[i:i+sequence_length:stride, :].values)
        y_list.append(data.iloc[i:i+output_length, -1])
    return np.array(X_list), np.array(y_list)

train_split = 0.7
n_train = int(train_split * len(resample_df))
n_test = len(resample_df) - n_train

features = ['day_of_week', 'hour_of_day', 'log_energy_consumption']
feature_array = resample_df[features].values



In [13]:
feature_array

array([[ 0.        , 17.        ,  4.00733319],
       [ 0.        , 18.        ,  5.17426472],
       [ 0.        , 19.        ,  5.15521652],
       ...,
       [ 4.        , 16.        ,  4.90527478],
       [ 4.        , 17.        ,  5.19295685],
       [ 4.        , 18.        ,  6.06378521]])

In [15]:
feature_array[:n_train]

array([[ 0.        , 17.        ,  4.00733319],
       [ 0.        , 18.        ,  5.17426472],
       [ 0.        , 19.        ,  5.15521652],
       ...,
       [ 5.        , 13.        ,  5.45246805],
       [ 5.        , 14.        ,  5.48755937],
       [ 5.        , 15.        ,  4.73033333]])

In [17]:
feature_array[:n_train, -1].reshape(-1, 1)

array([[4.00733319],
       [5.17426472],
       [5.15521652],
       ...,
       [5.45246805],
       [5.48755937],
       [4.73033333]])

In [18]:
# Fit Scaler only on Training features
feature_scaler = MinMaxScaler()
feature_scaler.fit(feature_array[:n_train])
# Fit Scaler only on Training target values
target_scaler = MinMaxScaler()
target_scaler.fit(feature_array[:n_train, -1].reshape(-1, 1))

# Transfom on both Training and Test data
scaled_array = pd.DataFrame(feature_scaler.transform(feature_array),
                            columns=features)



In [19]:
scaled_array

,day_of_week,hour_of_day,log_energy_consumption
0,0.000000,0.739130,0.216290
1,0.000000,0.782609,0.596809
2,0.000000,0.826087,0.590598
3,0.000000,0.869565,0.484000
4,0.000000,0.913043,0.421928
...,...,...,...
3285,0.666667,0.608696,0.416626
3286,0.666667,0.652174,0.324594
3287,0.666667,0.695652,0.509096
3288,0.666667,0.739130,0.602905


In [ ]:
X, y = create_sliding_window(scaled_array, 
                             sequence_length)

X_train = X[:n_train]
y_train = y[:n_train]

X_test = X[n_train:]
y_test = y[n_train:]

print(X_train.shape)
print(y_train.shape)

print(X_test.shape)
print(y_test.shape)

# Define Bayesian LSTM Architecture

To demonstrate a simple working example of the Bayesian LSTM, the model as defined in Uber's paper has been used a starting point. The network architecture is as follows:

Encoder-Decoder Stage:
 - A uni-directional LSTM with 2 stacked layers & 128 hidden units acting as an encoding layer to construct a fixed-dimension embedding state
 - A uni-directional LSTM with 2 stacked layers & 32 hidden units acting as a decoding layer to produce predictions at future steps
 - Dropout is applied at **both** training and inference for both LSTM layers


 Predictor Stage:
 - 1 fully-connected output layer with 1 output (for predicting the target value) to produce a single value for the target variable


By allowing dropout at both training and testing time, the model simulates random sampling, thus allowing varying predictions that can be used to estimate the underlying distribution of the target value, enabling explicit model uncertainties.


In [58]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable

class BayesianLSTM(nn.Module):

    def __init__(self, n_features, output_length, batch_size):

        super(BayesianLSTM, self).__init__()

        self.batch_size = batch_size # user-defined

        self.hidden_size_1 = 128 # number of encoder cells (from paper)
        self.hidden_size_2 = 32 # number of decoder cells (from paper)
        self.stacked_layers = 2 # number of (stacked) LSTM layers for each stage
        self.dropout_probability = 0.5 # arbitrary value (the paper suggests that performance is generally stable across all ranges)

        self.lstm1 = nn.LSTM(n_features, 
                             self.hidden_size_1, 
                             num_layers=self.stacked_layers,
                             batch_first=True)
        self.lstm2 = nn.LSTM(self.hidden_size_1,
                             self.hidden_size_2,
                             num_layers=self.stacked_layers,
                             batch_first=True)
        
        self.fc = nn.Linear(self.hidden_size_2, output_length)
        self.loss_fn = nn.MSELoss()
        
    def forward(self, x):
        batch_size, seq_len, _ = x.size()

        hidden = self.init_hidden1(batch_size)
        output, _ = self.lstm1(x, hidden)
        output = F.dropout(output, p=self.dropout_probability, training=True)
        state = self.init_hidden2(batch_size)
        output, state = self.lstm2(output, state)
        output = F.dropout(output, p=self.dropout_probability, training=True)
        output = output[:, -1, :] # take the last decoder cell's outputs
        y_pred = self.fc(output)
        return y_pred
        
    def init_hidden1(self, batch_size):
        hidden_state = Variable(torch.zeros(self.stacked_layers, batch_size, self.hidden_size_1))
        cell_state = Variable(torch.zeros(self.stacked_layers, batch_size, self.hidden_size_1))
        return hidden_state, cell_state
    
    def init_hidden2(self, batch_size):
        hidden_state = Variable(torch.zeros(self.stacked_layers, batch_size, self.hidden_size_2))
        cell_state = Variable(torch.zeros(self.stacked_layers, batch_size, self.hidden_size_2))
        return hidden_state, cell_state
    
    def loss(self, pred, truth):
        return self.loss_fn(pred, truth)

    def predict(self, X):
        return self(torch.tensor(X, dtype=torch.float32)).view(-1).detach().numpy()

### Begin Training

To train the Bayesian LSTM, we use the ADAM optimizer along with mini-batch gradient descent (`batch_size = 128`). For quick demonstration purposes, the model is trained for 150 epochs.

The Bayesian LSTM is trained on the first 70% of data points, using the aforementioned sliding windows of size 10. The remaining 30% of the dataset is held out purely for testing.

In [21]:
scaled_array

,day_of_week,hour_of_day,log_energy_consumption
0,0.000000,0.739130,0.216290
1,0.000000,0.782609,0.596809
2,0.000000,0.826087,0.590598
3,0.000000,0.869565,0.484000
4,0.000000,0.913043,0.421928
...,...,...,...
3285,0.666667,0.608696,0.416626
3286,0.666667,0.652174,0.324594
3287,0.666667,0.695652,0.509096
3288,0.666667,0.739130,0.602905


In [20]:
n_features = scaled_array.shape[-1]
n_features


3

In [ ]:
batch_size = 128
n_epochs = 1000
learning_rate = 0.0001

bayesian_lstm = BayesianLSTM(n_features=n_features,
                             output_length=output_length,
                             batch_size = batch_size)

criterion = torch.nn.MSELoss()
optimizer = torch.optim.Adam(bayesian_lstm.parameters(), lr=learning_rate)

In [64]:
bayesian_lstm.train()

for e in range(1, n_epochs+1):
    for b in range(0, len(X_train), batch_size):
        features = X_train[b:b+batch_size,:,:]
        target = y_train[b:b+batch_size]    

        X_batch = torch.tensor(features,dtype=torch.float32)    
        y_batch = torch.tensor(target,dtype=torch.float32)

        output = bayesian_lstm(X_batch)
        # print(output.shape)
        # print(y_batch.shape)
        loss = criterion(output.view(-1), y_batch.view(-1))  

        loss.backward()
        optimizer.step()        
        optimizer.zero_grad() 

    if e % 10 == 0:
      print('epoch', e, 'loss: ', loss.item())

epoch 10 loss:  0.03704504296183586
epoch 20 loss:  0.02226882427930832
epoch 30 loss:  0.01998351886868477
epoch 40 loss:  0.012195399031043053
epoch 50 loss:  0.011668379418551922
epoch 60 loss:  0.008281495422124863
epoch 70 loss:  0.00846160389482975
epoch 80 loss:  0.006129374727606773
epoch 90 loss:  0.009280135855078697
epoch 100 loss:  0.00588784646242857
epoch 110 loss:  0.008669695816934109
epoch 120 loss:  0.00511412275955081
epoch 130 loss:  0.003992131445556879
epoch 140 loss:  0.004649414215236902
epoch 150 loss:  0.0044736117124557495
epoch 160 loss:  0.006168636493384838
epoch 170 loss:  0.006428946740925312
epoch 180 loss:  0.004830840043723583
epoch 190 loss:  0.004268581513315439
epoch 200 loss:  0.00452776625752449
epoch 210 loss:  0.005733801051974297
epoch 220 loss:  0.0040904441848397255
epoch 230 loss:  0.002700401935726404
epoch 240 loss:  0.004869458731263876
epoch 250 loss:  0.00403921352699399
epoch 260 loss:  0.003512314520776272
epoch 270 loss:  0.00241463

KeyboardInterrupt: ignored

# Evaluating Model Performance

The Bayesian LSTM implemented is shown to produce reasonably accurate and sensible results on both the training and test sets, often comparable to other existing frequentist machine learning and deep learning methods.



In [65]:
offset = sequence_length

def inverse_transform(y):
  return target_scaler.inverse_transform(y.reshape(-1, output_length))

bayesian_lstm.eval()

testing_predictions = bayesian_lstm.predict(X_test)

testing_df = pd.DataFrame()
testing_df['date'] = resample_df['date'].iloc[n_train + offset::1] 
testing_df['log_energy_consumption'] = inverse_transform(testing_predictions)
testing_df['source'] = 'Test Prediction'

testing_truth_df = pd.DataFrame()
testing_truth_df['date'] = testing_df['date']
testing_truth_df['log_energy_consumption'] = inverse_transform(y_test)
testing_truth_df['source'] = 'True Values'

evaluation = pd.concat([testing_df,testing_truth_df], axis=0)

In [66]:
fig = px.line(evaluation.loc[evaluation['date'].between('2016-04-14', '2016-04-23')],
                 x="date",
                 y="log_energy_consumption",
                 color="source",
                 title="Log of Appliance Energy Consumption in Wh vs Time")
fig.show()